In [ ]:
!pip install nltk pysrt numpy pandas

In [ ]:
# Import necessary libraries
import pandas as pd
import pysrt
import pickle
import re
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Create final object
subtitles = {}

In [ ]:
# Gather all ads and phrases used to promote the subtitles platform
opensubs_ads = r'(Advertise your product or brand here)|(contact www\.OpenSubtitles\.(org|com) today)|(Support us and become VIP member)|(to remove all ads from www\.OpenSubtitles\.(org|com))|(-== \[ www\.OpenSubtitles\.(org|com) \] ==-)|(((Subtitles by )|(Sync by ))(.+?)$)|(font color=\"(.+)?\")|(Provided by(.+)$)|(^(https?):\/\/[^\s\/$.?#].[^\s]*$)|(Please rate this subtitle at (.)*$)|(Help other users to choose the best subtitles)'

In [ ]:
# Loop through files, take the text, remove all ads and add it to the object
for file in os.listdir():
  if not file.startswith("."):
    srt = pysrt.open(file)
    remove_ads = re.sub(re.compile(opensubs_ads), " ", srt.text)
    year = file.split("_")[-1].strip(".srt")
    title = "_".join(file.split("_")[:-1])
    subtitles[title] = [year, remove_ads]

In [ ]:
# Remove newline characters, braces ([{<>}]) and compact resulting whitespace 
for subtitle in subtitles:
  text = subtitles[subtitle][1]
  remove_newlines = re.sub(re.compile(r"\n+"), " ", text)
  remove_braces = re.sub(re.compile(r"(<\/?\w+(\s\w+=(.)+?)?>)|(\[.*?\])|\{.+?\}|\(.+?\)|♪"), " ", remove_newlines)
  compact_whitespace = re.sub(re.compile(r"\s+"), " ", remove_braces)
  subtitles[subtitle][1] = compact_whitespace


In [ ]:
df = pd.DataFrame.from_dict(subtitles, orient='index', columns=["Year", "Text"])
print(df.head())

                          Year                                               Text
Meet_the_Robinsons        2007  Then I didn't choose that one because it was g...
Saludos_Amigos            1942   Saludos Amigos A fond greeting to you A warm ...
Sleeping_Beauty           1959   I know you I waked with you once upon a dream...
The_Aristocats            1970   Which pet's address is the finest in Paris? W...
The_Emperor's_New_Groove  2000  Will you take a look at that? Pretty pathetic,...


In [37]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
def sentence_tokenize(txt: str):
    """Tokenize sentences: split text into sentences"""
    return sent_tokenize(txt)

In [ ]:
df['Sentence_Tokenized'] = df['Text'].apply(sentence_tokenize)
print(df.head())

                          Year  ...                                 Sentence_Tokenized
Meet_the_Robinsons        2007  ...  [Then I didn't choose that one because it was ...
Saludos_Amigos            1942  ...  [ Saludos Amigos A fond greeting to you A warm...
Sleeping_Beauty           1959  ...  [ I know you I waked with you once upon a drea...
The_Aristocats            1970  ...  [ Which pet's address is the finest in Paris?,...
The_Emperor's_New_Groove  2000  ...  [Will you take a look at that?, Pretty patheti...

[5 rows x 3 columns]


In [33]:
from numpy import array_split
df['50_chunks'] = df['Sentence_Tokenized'].apply(lambda x: array_split(x, 50))

In [36]:
df['Word_Tokenized'] = df['Text'].apply(word_tokenize)

50
